# Game of Graphs - Graph analytics on a GoT dataset

First, we need to run Memgraph. The easiest way to do this, is to run Memgraph Platform with Docker.
```
docker run -it -p 7687:7687 -p 7444:7444 -p 3000:3000 memgraph/memgraph-platform
```

In [221]:
from gqlalchemy import Memgraph
memgraph = Memgraph("127.0.0.1", 7687)

Let's see if there's anything in the database. To do that, we can count the number of nodes in the database with the following Cypher query:
```
MATCH (n) RETURN count(n) AS number_of_nodes;
```

With the GQLAlchemy, you can execute that query and fetch its results:

In [222]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))

{'number_of_nodes': 2677}


Besides simple Cypher query execution, you can also use GQLAlchemy query builder:

In [223]:
from gqlalchemy import match
results = match().node(variable="n").return_({"count(n)": "number_of_nodes"}).execute()
print(next(results))

{'number_of_nodes': 2677}


In the similar way, you can determine the number of relationships in the database:

In [224]:
results = memgraph.execute_and_fetch(
    """
    MATCH ()-[r]->() RETURN count(r) AS number_of_relationships ;
    """
)
print(next(results))

{'number_of_relationships': 11967}


And with the query builder like this:

In [225]:
from gqlalchemy import match
results = match().node().to(variable="r").node().return_({"count(r)": "number_of_relationships"}).execute()
print(next(results))

{'number_of_relationships': 11967}


The simplest way to load a dataset into Memgraph is by using **Memgraph Lab** which is currently running on `localhost:3000`. The first thing you're going to see on that address is the **Quick Connect**. You just have to click `Connect`!

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/2bb531a3459d5b07953ecb124b50ff677ee337bd/got-analysis/img/memgraph-lab-quick-connect.png" width="800" />

After you click `Connect`, head over to the `Datasets` tab, and load **Game of Thrones deaths** dataset.

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/2bb531a3459d5b07953ecb124b50ff677ee337bd/got-analysis/img/memgraph-lab-load-dataset.png" width="800" />

Now, let's again check the **number of nodes** and **relationships** in the database:

In [226]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))

{'number_of_nodes': 2677}


In [227]:
results = memgraph.execute_and_fetch(
    """
    MATCH ()-[r]->() RETURN count(r) AS number_of_relationships ;
    """
)
print(next(results))

{'number_of_relationships': 11967}


You can also visualize this in the Memgraph Lab, by running:

```
MATCH (u)-[r]->(m)
RETURN u, r, m;
```

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/da5a8dc96eb9035912bcf23f2ec9e457f2fbd262/got-analysis/img/memgraph-lab-got-dataset.png" width="800" />

Next, click on the **Graph Schema** tab and generate graph schema to see how the relationships are connecting the nodes in the database.

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/b483044ba9c88a4a43c7544d27af750bc84a36f8/got-analysis/img/memgraph-lab-graph-schema.png" width="800" />

Therefore, we have nodes labeled with `Character`, `Allegiance`, `Death`, `Episode`, `Location` and `Season`, and relationships of type `KILLED`, `LOYAL_TO`, `KILLER_IN`, `VICTIM_IN`, `HAPPENED_IN` and `PART_OF` connecting them. In more details, we have a whole graph model:

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/b483044ba9c88a4a43c7544d27af750bc84a36f8/got-analysis/img/graph-data-model.png" width="800" />

We can create a schema for this model inside the Python code by defining the classes that inherit from the `Node` and `Relationship` classes.

In [228]:
from gqlalchemy import Node, Relationship, Field

class Character(Node):
    name: str

class Allegiance(Node):
    name: str
        
class Death(Node):
    order: int
        
class Episode(Node):
    name: str
    number: int
    imdb_rating: float

class Season(Node):
    number: int

class Location(Node):
    name: str

class Killed(Relationship, type="KILLED"):
    method: str
    count: int

class LoyalTo(Relationship, type="LOYAL_TO"):
    pass

class VictimIn(Relationship, type="VICTIM_IN"):
    pass

class KillerIn(Relationship, type="KILLER_IN"):
    pass

class HappenedIn(Relationship, type="HAPPENED_IN"):
    pass

class PartOf(Relationship, type="PART_OF"):
    pass

Since the data is already loaded into Memgraph, there is no need to define any constraints. If you want to learn how to do that, head over to the [GQLAlchemy docs](https://memgraph.com/docs/gqlalchemy/how-to-guides). What’s going on here:

- `Node` is a Python class that maps to a graph object in `Memgraph`.
- Classes that inherit from `Node` map to a single label in the graph database.
- In this case, the class `Character` maps to the label `:Character`.
- The property `name` is mapped to property of the node labeled `Character` in the graph database.
- Relationships can also have properties, such as defined in `Killed` class that maps to `:KILLED` type of the relationship in the database.
- If the relationship doesn't have a property, you can just use `pass`.

## Simple analytics with GQLAlchemy

Let's first list the total number of seasons and episodes within each season in the dataset:

In [229]:
results = memgraph.execute_and_fetch(
    """
    MATCH (s:Season)
    RETURN count(s) AS total_number_of_seasons;
    """
)

print(next(results))

{'total_number_of_seasons': 8}


In [230]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[:PART_OF]->(s:Season)
    RETURN s, collect(e) AS episodes
    ORDER BY s.number;
    """
)

for result in results:
    print("Season:", result["s"].number, "Number of episodes:", len(result["episodes"]))

Season: 1 Number of episodes: 9
Season: 2 Number of episodes: 9
Season: 3 Number of episodes: 9
Season: 4 Number of episodes: 10
Season: 5 Number of episodes: 10
Season: 6 Number of episodes: 9
Season: 7 Number of episodes: 7
Season: 8 Number of episodes: 6


Let's also see how many characters and deaths there are in the dataset:

In [231]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)
    RETURN count(c) AS total_number_of_characters;
    """
)

print(next(results))

{'total_number_of_characters': 289}


In [232]:
results = memgraph.execute_and_fetch(
    """
    MATCH (d:Death)
    RETURN count(d) AS total_number_of_deaths;
    """
)

print(next(results))

{'total_number_of_deaths': 2224}


## No deaths episodes

We all know that Game of Thones is famous for the amount of (important) people that died throughout the TV show. But, there are couple of exceptions - episode when no one died! Above we listed the number of episodes per season that are in the dataset, and that leads us to the conclusion on how many episodes have no deaths in them. Let's first create lists of all episodes per season and then filter it out to find out which ones are missing.

In [233]:
season_1 = {'Winter Is Coming ', 'The Kingsroad ', 'Lord Snow', 
            'Cripples Bastards and Broken Things ', 'The Wolf and the Lion ', 
            'A Golden Crown ', 'You Win or You Die', 'The Pointy End ', 
            'Baelor ', 'Fire and Blood'}

season_2 = {"The North Remembers", "The Night Lands ", "What Is Dead May Never Die", 
            "Garden of Bones", "The Ghost of Harrenhal ", "The Old Gods and the New", 
            "A Man Without Honor", "The Prince of Winterfell", "Blackwater ", "Valar Morghulis "}

season_3 = {"Valar Dohaeris ", "Dark Wings Dark Words ", "Walk of Punishment ", 
            "And Now His Watch Is Ended ", "Kissed by Fire", "The Climb ", "The Bear and the Maiden Fair", 
            "Second Sons ", "The Rains of Castamere", "Mhysa "}

season_6 = {"The Red Woman ", "Home ", "Oathbreaker ", "Book of the Stranger ", "The Door ", 
            "Blood of My Blood", "The Broken Man ", "No One", "Battle of the Bastards", "The Winds of Winter"}

First, let's check which episode is missing from the Season 1:

In [234]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[p:PART_OF]->(s:Season {number: 1})
    RETURN e;
    """
)

season_1_deaths = []

for result in results:
    season_1_deaths.append(result["e"].name)

set_season_1_deaths = set(season_1_deaths)
missing = list(sorted(set_season_1 - set_season_1_deaths))
print(missing)

['Lord Snow']


Looks like it's the episode **Lord Snow**, which is the Episode 3. Next, let's see which episode is missing from Season 2:

In [235]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[p:PART_OF]->(s:Season {number: 2})
    RETURN e;
    """
)
season_2_deaths = []
for result in results:
    season_2_deaths.append(result["e"].name)

set_season_2_deaths = set(season_2_deaths)
missing_2 = list(sorted(set_season_2 - set_season_2_deaths))
print(missing_2)

['The Prince of Winterfell']


The episode that is missing is called **The Prince of Winterfell** and it is Episode 8. Next up, Season 3:

In [236]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[p:PART_OF]->(s:Season {number: 3})
    RETURN e;
    """
)
season_3_deaths = []
for result in results:
    season_3_deaths.append(result["e"].name)

set_season_3_deaths = set(season_3_deaths)
missing_3 = list(sorted(set_season_3 - set_season_3_deaths))
print(missing_3)

['The Bear and the Maiden Fair']


**The Bear and the Maiden Fair** is the episode without any killings, and it's 7th Episode of Season 3. The last season which we have to check is Season 6:

In [237]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[p:PART_OF]->(s:Season {number: 6})
    RETURN e;
    """
)
season_6_deaths = []
for result in results:
    season_6_deaths.append(result["e"].name)

set_season_6_deaths = set(season_6_deaths)
missing_6 = list(sorted(set_season_6 - set_season_6_deaths))
print(missing_6)

['Blood of My Blood']


That's it! **Blood of My Blood** is the last episode without any deaths, and it is the Episode 6!
Besided that, there are couple of sources claming that there are more episodes without any deaths: Season 3 - Episode 1 (**Valar Dohaeris**), Season 3 - Episode 2 (**Dark Wings, Dark Words**), and Season 8 - Episode 2 (**A Knight of the Seven Kingdoms**). Let's check whether that's true!

### Valar Dohaeris

In [238]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)-[v:VICTIM_IN]->(d:Death)
    -[:HAPPENED_IN]->(e:Episode {number: 1})
    -[p:PART_OF]->(s:Season {number: 3})
    RETURN c;
    """
)

print(next(results)["c"].name)

Manticore


<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/a698d3095c5bc0066a379004ef88c2a44f8c8821/got-analysis/img/memgraph-lab-manticore.png" width="800" />

### Dark Wings, Dark Words

In [239]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)-[v:VICTIM_IN]->(d:Death)
    -[:HAPPENED_IN]->(e:Episode {number: 2})
    -[p:PART_OF]->(s:Season {number: 3})
    RETURN c;
    """
)

print(next(results)["c"].name)

Hoster Tully


<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/a698d3095c5bc0066a379004ef88c2a44f8c8821/got-analysis/img/memgraph-lab-hoster-tully.png" width="800" />

### A Knight of the Seven Kingdoms

In [240]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)-[v:VICTIM_IN]->(d:Death)
    -[:HAPPENED_IN]->(e:Episode {number: 2})
    -[p:PART_OF]->(s:Season {number: 8})
    RETURN c;
    """
)

print(next(results)["c"].name)

Ned Umber


<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/a698d3095c5bc0066a379004ef88c2a44f8c8821/got-analysis/img/memgraph-lab-ned-umber.png" width="800" />

## Characters that killed themselves

There were a couple of characters that managed to run away from the other killers, but still ended up dead. Let's see how:

In [241]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)-[k:KILLED]->(d:Character)
    WHERE c = d
    RETURN c, k;
    """
)

for result in results:
    print("Character:", result["c"].name, "| Method:", result["k"].method)

Character: Dothraki man | Method: Arakh
Character: Sandor the Hound Clegane | Method: Dragonfire (Dragon)
Character: Melisandre the Red Woman of Asshai | Method: Old Age
Character: Bolton soldier | Method: Arrow
Character: Jaqen Hghar | Method: Poison
Character: Peasant | Method: Poison
Character: Greyjoy Soldier | Method: Sword
Character: Greyjoy Soldier | Method: Axe
Character: Greyjoy Soldier | Method: Mace
Character: Greyjoy Soldier | Method: Knife
Character: Tommen Baratheon | Method: Falling
Character: Sons of the Harpy agent | Method: Sword
Character: Aemon Targaryen | Method: Old Age
Character: Gladiator | Method: Sword
Character: Gladiator | Method: Spear
Character: Selyse Florent | Method: Rope


I guess the only ones who died of old age are **Melisandre** and **Aemon Targaryen**. But, let's be real - Aemon is the only true badass here!

## Graph traversals and PageRank

Let's see how the importance of episodes and locations in the dataset is truly measured by the number of deaths. First, we can list the top 10 locations where the most deaths occurred:

In [242]:
results = memgraph.execute_and_fetch(
    """
    MATCH (l:Location)<-[:HAPPENED_IN]-(d:Death)
    RETURN l AS location, count(d) AS death_count
    ORDER BY death_count DESC
    LIMIT 10;
    """
)

for result in results:
    print(result["location"].name, result["death_count"])

Kings Landing 1170
Roseroad 207
Winterfell 169
Meereen 136
The Twins 89
Beyond the Wall 78
Castle Black 66
The Narrow Sea 35
Riverlands 28
The Wall 26


We can also run PageRank algorithm as a measure of node influence in a network:

In [243]:
results = memgraph.execute_and_fetch(
    """
    CALL pagerank.get()
    YIELD *
    WITH node, rank
    WHERE node:Location
    RETURN node, rank
    ORDER BY rank DESC
    LIMIT 10;
    """
)

for result in results:
    print("Episode:", result["node"].name, "| Rank:", result["rank"])

Episode: Kings Landing | Rank: 0.06106011381918924
Episode: Roseroad | Rank: 0.010786290904476399
Episode: Winterfell | Rank: 0.009650604216610618
Episode: Meereen | Rank: 0.007653449136907163
Episode: The Twins | Rank: 0.005029859049634092
Episode: Beyond the Wall | Rank: 0.0048574484597048535
Episode: Castle Black | Rank: 0.003955983097311168
Episode: The Narrow Sea | Rank: 0.0021040262793516795
Episode: Riverlands | Rank: 0.0019043870463720024
Episode: The Wall | Rank: 0.001583596860766157


And which episodes have the most deaths?

In [244]:
results = memgraph.execute_and_fetch(
    """
    MATCH (d:Death)-[:HAPPENED_IN]->(e:Episode)
    RETURN e AS episode, count(d) AS death_count
    ORDER BY death_count DESC
    LIMIT 10;
    """
)

for result in results:
    print(result["episode"].name, result["death_count"])

The Bells  844
The Spoils of War  205
The Winds of Winter 203
Battle of the Bastards 133
The Watchers on the Wall  86
Blackwater  72
The Dance of Dragons 58
Dragonstone  54
Sons of the Harpy 48
The Rains of Castamere 45


Let's check whether the **PageRank algorithm** gives us the similiar result:

In [245]:
results = memgraph.execute_and_fetch(
    """
    CALL pagerank.get()
    YIELD *
    WITH node, rank
    WHERE node:Episode
    RETURN node, rank
    ORDER BY rank DESC
    LIMIT 10;
    """
)

for result in results:
    print("Episode:", result["node"].name, "| Rank:", result["rank"])

Episode: The Bells  | Rank: 0.043164724601018695
Episode: The Winds of Winter | Rank: 0.010801371244279762
Episode: The Spoils of War  | Rank: 0.010652824071617746
Episode: Battle of the Bastards | Rank: 0.007326424536668109
Episode: The Watchers on the Wall  | Rank: 0.004888653411156207
Episode: Blackwater  | Rank: 0.0040451560244256475
Episode: The Dance of Dragons | Rank: 0.003304325314698726
Episode: Dragonstone  | Rank: 0.002979021268992005
Episode: Sons of the Harpy | Rank: 0.002835536547706026
Episode: The Rains of Castamere | Rank: 0.002811794141977363


## Seasons and allegiances

### Number of kills per season

In [246]:
results = memgraph.execute_and_fetch(
    """
    MATCH (d:Death)-[:HAPPENED_IN]->(s:Season)
    RETURN s AS season, count(d) AS death_count
    ORDER BY death_count DESC;
    """
)

for result in results:
    print(result["season"].number, result["death_count"])

8 892
6 396
7 334
4 170
5 158
2 130
3 86
1 58


### Top seasons by their IMDB rating

In [247]:
results = memgraph.execute_and_fetch(
    """
    MATCH (e:Episode)-[:PART_OF]->(s:Season)
    RETURN s AS season, round(100 * avg(e.imdb_rating))/100 AS rating
    ORDER BY rating DESC;
    """
)

for result in results:
    print("Season:", result["season"].number, "| Rating:", result["rating"])

Season: 4 | Rating: 9.31
Season: 1 | Rating: 9.14
Season: 6 | Rating: 9.13
Season: 7 | Rating: 9.1
Season: 3 | Rating: 9.09
Season: 2 | Rating: 8.98
Season: 5 | Rating: 8.83
Season: 8 | Rating: 6.38


### Top 10 allegiances by the kill/death ratio (KDR)

In [248]:
results = memgraph.execute_and_fetch(
    """
    MATCH (:Character)-[death:KILLED]->(:Character)-[:LOYAL_TO]->(a:Allegiance)
    WITH a, sum(death.count) AS deaths
    MATCH (:Character)<-[kill:KILLED]-(:Character)-[:LOYAL_TO]->(a)
    RETURN a AS allegiance,
           sum(kill.count) AS kills,
           deaths,
           round(100 *(tofloat(sum(kill.count))/deaths))/100 AS KDR
    ORDER BY KDR DESC
    LIMIT 10;
    """
)

for result in results:
    print("Allegiance:", result["allegiance"].name, "| Kills:", result["kills"], 
          "| Deaths:", result["deaths"], "| KDR:", result["KDR"])

Allegiance: White Walkers | Kills: 205 | Deaths: 5 | KDR: 41.0
Allegiance: Warlocks of Qarth | Kills: 11 | Deaths: 1 | KDR: 11.0
Allegiance: House Clegane | Kills: 54 | Deaths: 5 | KDR: 10.8
Allegiance: House Targaryen | Kills: 1329 | Deaths: 151 | KDR: 8.8
Allegiance: Lord of Light | Kills: 7 | Deaths: 1 | KDR: 7.0
Allegiance: House Umber | Kills: 7 | Deaths: 2 | KDR: 3.5
Allegiance: Faceless Men | Kills: 3 | Deaths: 1 | KDR: 3.0
Allegiance: House Baratheon of Kings Landing | Kills: 194 | Deaths: 66 | KDR: 2.94
Allegiance: Nights Watch | Kills: 394 | Deaths: 143 | KDR: 2.76
Allegiance: Sand Snakes | Kills: 8 | Deaths: 3 | KDR: 2.67


### Battle of Bastards causalties - Starks vs Boltons

In [249]:
results = memgraph.execute_and_fetch(
    """
    MATCH (c:Character)-[:LOYAL_TO]->(a:Allegiance)
    MATCH (c)-[:VICTIM_IN]-(d:Death)-[:HAPPENED_IN]-(:Episode {name: 'Battle of the Bastards'})
    RETURN a AS house, count(d) AS death_count
    ORDER BY death_count DESC
    LIMIT 2;
    """
)

for result in results:
    print("Allegiance:", result["house"].name, "| Deaths:", result["death_count"])

Allegiance: House Bolton | Deaths: 49
Allegiance: House Stark | Deaths: 42


## Is Daenerys that bad?

Let's list all characters Daenerys killed and count how many of them, along with the episode that happened in:

In [250]:
results = memgraph.execute_and_fetch(
    """
    MATCH (daenerys:Character {name: 'Daenerys Targaryen'})-[:KILLED]->(victim:Character)
    MATCH (daenerys)-[:KILLER_IN]->(d:Death)<-[:VICTIM_IN]-(victim)
    MATCH (d)-[:HAPPENED_IN]-(e:Episode)
    RETURN DISTINCT victim, count(d) AS kill_count, e AS episode
    ORDER BY kill_count DESC;
    """
)

for result in results:
    print("Victim:", result["victim"].name, "| Kill Count:", result["kill_count"]
          , "| Episode name:", result["episode"].name)

Victim: Golden Company soldier | Kill Count: 374 | Episode name: The Bells 
Victim: Kings Landing Citizen | Kill Count: 227 | Episode name: The Bells 
Victim: Lannister soldier | Kill Count: 178 | Episode name: The Spoils of War 
Victim: Lannister soldier | Kill Count: 166 | Episode name: The Bells 
Victim: Sons of the Harpy agent | Kill Count: 54 | Episode name: The Dance of Dragons
Victim: Dothraki Khal | Kill Count: 14 | Episode name: Book of the Stranger 
Victim: The Masters Soldier | Kill Count: 12 | Episode name: Battle of the Bastards
Victim: Greyjoy Soldier | Kill Count: 2 | Episode name: The Bells 
Victim: Horse | Kill Count: 2 | Episode name: The Spoils of War 
Victim: Xaro Xhoan Daxos | Kill Count: 1 | Episode name: Valar Morghulis 
Victim: Khal Moro | Kill Count: 1 | Episode name: Book of the Stranger 
Victim: Golden Company horse | Kill Count: 1 | Episode name: The Bells 
Victim: Pyat Pree | Kill Count: 1 | Episode name: Valar Morghulis 
Victim: Zalla | Kill Count: 1 | Epi

### Who is the killer influencer?

In [251]:
results = memgraph.execute_and_fetch(
    """
    CALL betweenness_centrality.get(False)
    YIELD node, betweenness_centrality
    WITH node, betweenness_centrality
    WHERE 'Character' IN labels(node)
    RETURN node, betweenness_centrality
    ORDER BY betweenness_centrality DESC
    LIMIT 10;
    """
)

for result in results:
    print(result["node"].name, "| BC:", result["betweenness_centrality"])

Daenerys Targaryen | BC: 0.26416716859707734
Lannister soldier | BC: 0.11163138590238249
Sons of the Harpy agent | BC: 0.04326120746568803
Jon Snow | BC: 0.04152733973021267
Wildling | BC: 0.03782396045716545
Stark Soldier | BC: 0.028609456095599384
Baratheon of Dragonstone soldier | BC: 0.027533816622595356
Cersei Lannister | BC: 0.026445092829209982
Stark soldier | BC: 0.02432818459201447
Nights Watch brother | BC: 0.022719177729567424


## You know nothing, Jon Snow

### Who would survive if Jon Snow stayed dead?

In [252]:
results = memgraph.execute_and_fetch(
    """
    MATCH (jon:Character {name: 'Jon Snow'})-[:KILLED]->(victim:Character)
    MATCH (jon)-[:VICTIM_IN]->(jon_death:Death)
    MATCH (jon)-[:KILLER_IN]->(victim_death:Death)<-[:VICTIM_IN]-(victim)
    WHERE victim_death.order > jon_death.order
    RETURN DISTINCT victim, count(victim_death) AS kill_count
    ORDER BY kill_count DESC;
    """
)

for result in results:
    print("Victim:", result["victim"].name, "| Kill count:", result["kill_count"])

Victim: Bolton soldier | Kill count: 20
Victim: Lannister soldier | Kill count: 7
Victim: Daenerys Targaryen | Kill count: 1
Victim: Olly | Kill count: 1
Victim: Stark Soldier | Kill count: 1
Victim: Alliser Thorne | Kill count: 1
Victim: Othell Yarwyck | Kill count: 1
Victim: Bowen Marsh | Kill count: 1


### Who killed Jon Snow?

In [253]:
results = memgraph.execute_and_fetch(
    """
    MATCH (jon:Character {name: 'Jon Snow'})-[v:VICTIM_IN]
    ->(d:Death)<-[k:KILLER_IN]-(c:Character)
    WITH jon, v, d, k, c
    MATCH (c)-[v2:VICTIM_IN]->(d2:Death)<-[k2:KILLER_IN]-(c2:Character)
    RETURN c, jon;
    """
)

for result in results:
    print(result["c"].name, "killed", result["jon"].name, "and", 
          result["jon"].name, "killed", result["c"].name)

Olly killed Jon Snow and Jon Snow killed Olly


<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/582630e6240bec61876192174d1e746ef809a788/got-analysis/img/memgraph-lab-jon-snow-olly.png" width="800" />

### Who is the biggest traitor?

In [254]:
results = memgraph.execute_and_fetch(
    """
    MATCH (killer:Character)-[:KILLED]->(victim:Character)
    MATCH (killer)-[:LOYAL_TO]->(a:Allegiance)<-[:LOYAL_TO]-(victim)
    RETURN killer AS traitor, count(victim) AS kill_count
    ORDER BY kill_count DESC
    LIMIT 10;
    """
)

for result in results:
    print("Traitor:", result["traitor"].name, "| Kill count:", result["kill_count"])


Traitor: Jon Snow | Kill count: 9
Traitor: Ramsay Bolton | Kill count: 4
Traitor: Theon Greyjoy | Kill count: 4
Traitor: Sandor the Hound Clegane | Kill count: 3
Traitor: Reek | Kill count: 2
Traitor: Euron Greyjoy | Kill count: 2
Traitor: Daario Naharis | Kill count: 2
Traitor: Gregor the Mountain Clegane | Kill count: 2
Traitor: Daenerys Targaryen | Kill count: 2
Traitor: Tyrion Lannister | Kill count: 2


## Dijkstra killing it

Memgraph supports graph algorithms as well. Let's use **Dijkstra's shortest path algorithm** to show the most gruesome path of kills. An example kill path is: Jon Snow killed 5 Lannister Soldiers and they killed 10 Stark soldiers with total kill_count of 15.

In [255]:
results = memgraph.execute_and_fetch(
    """
    MATCH p = (:Character)-[:KILLED * wShortest (e,v | e.count) kill_count]->(:Character)
    RETURN nodes(p) AS kill_list, kill_count
    ORDER BY kill_count DESC
    LIMIT 1;
    """
)

for result in results:
    for kill in result["kill_list"]:
        print(kill.properties["name"])

Olly
Jon Snow
Daenerys Targaryen
Golden Company soldier


That's it! Hopefully, you gained new knowledge and enjoyed this workshop. If you have any questions, don't hesitate to contact me directly or [join our Discord server](https://discord.gg/memgraph).